<a href="https://colab.research.google.com/github/Zarasim/building-damage-assessment/blob/main/model_training_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/Zarasim/building-damage-assessment

Cloning into 'building-damage-assessment'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 275 (delta 69), reused 144 (delta 40), pack-reused 98 (from 1)
Receiving objects: 100% (275/275), 55.13 MiB | 26.85 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [3]:
%ls

building-damage-assessment/  drive/  sample_data/


In [4]:
%cp drive/MyDrive/Data/yolo.zip building-damage-assessment/

In [5]:
%cd building-damage-assessment

/content/building-damage-assessment


In [6]:
%mkdir /content/building-damage-assessment/training_output

In [7]:
!pip install -r requirements.txt

Obtaining file:///content/building-damage-assessment (from -r requirements.txt (line 12))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.6/646.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.7 MB/s eta 0:0

In [9]:
!nvidia-smi

Thu Sep 12 18:40:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
pip install torch torchvision torchaudio

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.6 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.0+cu121
    Uninstalling torchaudio-2.4.0+cu121:
      Successfully uninstalled torchaudio-2.4.0+cu121


In [11]:
# check GPU is available
import torch
torch.cuda.is_available()

True

In [12]:
!unzip yolo.zip

Archive:  yolo.zip
   creating: yolo/
   creating: yolo/dataset/
   creating: yolo/dataset/images/
   creating: yolo/dataset/images/train/
  inflating: yolo/dataset/images/train/socal-fire_00000429_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-matthew_00000089_post_disaster.png  
  inflating: yolo/dataset/images/train/socal-fire_00000253_post_disaster.png  
  inflating: yolo/dataset/images/train/guatemala-volcano_00000001_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-florence_00000002_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-matthew_00000155_post_disaster.png  
  inflating: yolo/dataset/images/train/santa-rosa-wildfire_00000076_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-michael_00000258_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-harvey_00000008_post_disaster.png  
  inflating: yolo/dataset/images/train/hurricane-florence_00000374_post_disaster.png  
  inflating:

In [13]:
import yaml
with open("yolo/dataset/data.yaml", 'r') as stream:
    try:
        config = yaml.safe_load(stream)
        config["train"] = "/content/building-damage-assessment/yolo/dataset/images/train"
        config["val"] = "/content/building-damage-assessment/yolo/dataset/images/val"
        config["test"] = "/content/building-damage-assessment/yolo/dataset/images/test"
        with open("yolo/dataset/data.yaml", 'w') as outfile:
            yaml.dump(config, outfile, default_flow_style=False)
    except yaml.YAMLError as exc:
        print(exc)

In [14]:
!python src/yolo_damage_assessment/train_yolo.py --config config/config.yaml --output /content/building-damage-assessment/training_output

2024-09-12 18:41:18,401 - root - INFO - Starting model training
2024-09-12 18:41:18,403 - root - INFO - Loaded configuration from config/config.yaml
2024-09-12 18:41:18,403 - root - INFO - Set CUDA environment variable
2024-09-12 18:41:18,408 - root - INFO - CUDA available: True
100% 21.5M/21.5M [00:00<00:00, 216MB/s]
2024-09-12 18:41:19,417 - root - INFO - Initialized YOLO model: yolov8s.pt
2024-09-12 18:41:19,417 - root - INFO - Starting model training
New https://pypi.org/project/ultralytics/8.2.92 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.221 🚀 Python-3.10.12 torch-2.1.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=./yolo/dataset/data.yaml, epochs=10, patience=50, batch=8, imgsz=[640, 640], save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=building_damage_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=Fal